In [144]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

from src.model import UNet
from src.dataset import get_load_data
from src.loss_function import one_hot_encode

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [176]:
unet_model = UNet(num_classes=21)
train, test = get_load_data(root = "../../data", dataset = "VOCSegmentation", download = False)  
img, smnt = train[0] 
img = img.reshape(1, 3, 572, 572)

In [177]:
import torch
import numpy as np
smnt = smnt.resize((388, 388))
smnt = torch.tensor(np.asarray(smnt), dtype = torch.float32).reshape(1, 1, 388, 388)
# smnt = one_hot_encode(smnt, 20)
pred = unet_model(img)

In [75]:
from torch.nn import Softmax
m = Softmax(dim = 0)
m(pred[0,:, 0,0])

tensor(1.0000, grad_fn=<AddBackward0>)

In [66]:
pred_argmax = pred.argmax(dim = 1) + 1

In [58]:
pred_argmax = pred_argmax.type(torch.float32)

In [ ]:
loss = torch.nn.CrossEntropyLoss(reduction='none')(pred_argmax, smnt)
loss = torch.mean(loss)

In [179]:
smnt.shape

torch.Size([1, 1, 388, 388])

In [185]:
pred_permute.squeeze(0).shape

torch.Size([388, 388, 21])

In [190]:
smnt.squeeze(1).shape

torch.Size([1, 388, 388])

In [203]:
smnt.dtype != torch.int64

False

In [198]:
loss = torch.nn.CrossEntropyLoss(ignore_index=255)
smnt = smnt.type(torch.int64)
loss(pred, smnt.squeeze(1))

tensor(3.1264, grad_fn=<NllLoss2DBackward0>)